In [7]:
import numpy as np

In [74]:
class Graph:
    def __init__(self,adj={}):
        self.adj = adj
        self.nodes = []
        self.nodes = list(adj)
        for n in self.adj:
            self.nodes += list(self.adj[n])
        self.nodes = list(set(self.nodes))

        for n in self.nodes:
            if n not in self.adj:
                self.adj[n] = {}

class entry:
    def __init__(self,item='',key=''):
        self.key = key
        self.item = item
                
                
class heap:
    def __init__(self,max=False):
        self.Q = []
        self.item_index = {}
        self.max = max
    
    def parent(self,i):
        if i==0: return 0
        return int(np.floor((i-1)/2))
    
    def children(self,i):
        return 2*i+1, 2*(i+1)
    
    def swap(self,i,j):
        e_i = self.Q[i]
        e_j = self.Q[j]
        self.Q[i] = e_j
        self.Q[j] = e_i
        self.item_index[e_j.item] = i
        self.item_index[e_i.item] = j
    
    def push(self,item,key):
        if self.max:
            key = -key

        self.Q.append(entry(item=item,key=key))
        self.item_index[item] = len(self.Q)-1
        self.bubble_up(len(self.Q)-1)
        
    def pop(self):
        
        if len(self.Q)==1:
            e = self.Q.pop()
            self.item_index.pop(e.item)
        else:
            e = self.Q[0]
            self.item_index.pop(e.item)
            self.Q[0] = self.Q.pop()
            self.item_index[self.Q[0].item] = 0
            self.bubble_down(0)
        if self.max:
            e.key = -e.key
        return e    
    
    def bubble_up(self,i):
        c_idx = i
        p_idx = self.parent(c_idx)
        
        while self.Q[p_idx].key > self.Q[c_idx].key:
            self.swap(p_idx,c_idx)
            c_idx = p_idx
            p_idx = self.parent(c_idx)    
   
    def bubble_down(self,i):
        
        p_idx = i
        c1, c2 = self.children(p_idx)
        
        while ((c1 < len(self.Q) and self.Q[p_idx].key > self.Q[c1].key) or 
               (c2 < len(self.Q) and self.Q[p_idx].key > self.Q[c2].key)):
            if c1 < len(self.Q) and c2 < len(self.Q):
                if self.Q[c1].key < self.Q[p_idx].key and self.Q[c2].key >= self.Q[p_idx].key:
                    self.swap(p_idx,c1)
                    p_idx = c1
                elif (self.Q[c2].key < self.Q[p_idx].key) and (self.Q[c1].key >= self.Q[p_idx].key):
                    self.swap(p_idx,c2)
                    p_idx = c2
                else:
                    if self.Q[c2].key < self.Q[c1].key:
                        self.swap(p_idx,c2)
                        p_idx = c2
                    else:
                        self.swap(p_idx,c1)
                        p_idx = c1
            elif c1 < len(self.Q):
                self.swap(p_idx,c1)
                p_idx = c1
            else:
                self.swap(p_idx,c2)
                p_idx = c2
            c1, c2 = self.children(p_idx)
            
    def maximum(self):
        if self.is_empty():
            return np.inf
        else:
            return -self.Q[0].key
    
    def minimum(self):
        if self.is_empty():
            return -np.inf
        else:
            return self.Q[0].key
        
    def update(self,item,key):
        
        if self.max:
            key = -key
        
        idx = self.item_index[item]
        c1, c2 = self.children(idx)
        p_idx = self.parent(idx) 
        
        if (idx == 0):
            if (key > self.Q[c1].key or key > self.Q[c2].key):
                self.bubble_down(idx)
        
        elif (idx == len(self.Q)-1):
            if (key < self.Q[p_idx].key):
                self.bubble_up(idx)
        else:
            if ((c1 < len(self.Q) and key > self.Q[c1].key) or 
               (c2 < len(self.Q) and key > self.Q[c2].key)):
                self.bubble_down(idx)
            elif (key < self.Q[p_idx].key):
                self.bubble_up(idx)

    def is_empty(self):
        if self.Q:
            return False
        return True
              

In [48]:
def dijkstra(g,ith_node):
    dist = {}
    dist[ith_node] = 0
    h = heap()
    
    for v in g.nodes:
        if v != ith_node:
            dist[v] = np.inf
        h.push(v,dist[v])

    while not h.is_empty():
        u = h.pop().item
        for v in g.adj[u]:
            if v in h.item_index:
                alt = dist[u] + g.adj[u][v]
                if alt < dist[v]:
                    h.update(v,alt)
                    dist[v] = alt                
                    
    return dist

**Dijkstra algorithm**

In [25]:
f = open('dijkstraData.txt','r')
lines = f.readlines()

In [358]:
nodes = {}
for line in lines:
    tmp = line.strip('\t\n').split('\t')
    nodes[int(tmp[0])] = {}
    for pair in tmp[1:]:
        [v,l] = pair.split(',')
        nodes[int(tmp[0])][int(v)] = int(l)

In [359]:
g = Graph(nodes)

In [360]:
distances = dijkstra(g,1)

In [361]:
arr = [7,37,59,82,99,115,133,165,188,197]
entry = []
for a in arr:
    entry.append(distances[a])  
print(entry)

[2599, 2610, 2947, 2052, 2367, 2399, 2029, 2442, 2505, 3068]


**Median Maintenance**

In [75]:
f = open('Median.txt','r')
lines = f.readlines()
lines = [int(l.strip('\n')) for l in lines]

In [76]:
hlow = heap(max=True)
hhigh = heap(max=False)

medians = []

for a in lines:
    if a < hlow.maximum():
        hlow.push(a,a)
    else:
        hhigh.push(a,a)
    if len(hlow.Q)==(2+len(hhigh.Q)):
        e = hlow.pop()
        hhigh.push(e.item,e.key)
    if len(hhigh.Q)==(2+len(hlow.Q)):
        e = hhigh.pop()
        hlow.push(e.item,e.key)
    
    if len(hhigh.Q)>len(hlow.Q):
        m = hhigh.minimum()
    elif len(hlow.Q)>len(hhigh.Q):
        m = hlow.maximum()
    else:
        m = hlow.maximum()
    medians.append(m)    

In [77]:
print(sum(medians) % 10000)

1213
